In [1]:
import pandas as pd
import numpy as np

1、最终目的获取终端用户的信息描述：如猜测对方性别比重、年龄段比重等；

2、原始数据“userdata.txt”的简单说明（感觉手机号是Base64方式编码的）：
  1）索引0：手机号（MSISDN）；
  2）索引11：使用日期（长整型的值）；
  3）索引12：使用时长；
  4）索引15：appid（对应appTab.txt里的appid，注意有的值前面补0了）；

3、数据“userdata_new.txt”是我处理后的数据：模拟手机号、appid、使用日期、使用时长；

In [2]:
app_tab = pd.read_csv('data/appTab.txt',encoding='gb18030',sep='|',names=['app_id','name','male','female','_24','25_30','31_35','36_40','40_'])
app_tab = app_tab.set_index('app_id',drop=False)



In [3]:
user_data = pd.read_csv('data/userdata_new.txt',sep='|',names = ['tel','app_id','date','time'])
user_data = user_data.set_index('tel')

In [4]:
# app_tab

In [5]:
user_data.head()

,app_id,date,time
tel,,,
13900000001,50057,1471020386250,70
13900000002,50057,1471017860712,75
13900000003,10005,1471018898111,161
13900000004,10005,1471020522545,107
13900000005,10005,1471020528891,20596


In [6]:
def get_name(id):
    return app_tab.loc[id,'name']

In [7]:
from pyecharts import Pie
def gender_rate(app_tab,app_id):
    gender_data = app_tab.loc[[app_id],['male','female']]
    gender_data = np.array(gender_data).flat
    attr = ['male','female']
    v1 = gender_data
    pie = Pie(get_name(app_id)+u"男女比例")
    pie.add("", attr, v1, is_label_show=True)
    return pie

In [8]:
gender_rate(app_tab,10003)

In [9]:
from pyecharts import Pie
def age_distribution(app_tab,app_id):
    age_data = app_tab.loc[[app_id],['_24','25_30','31_35','36_40','40_']]
    age_data = np.array(age_data).flat
    attr = ['24岁以下','25-30岁','31-35岁','36-40岁','40岁以上']
    v1 = age_data
    pie = Pie(get_name(app_id)+u"年龄分布")
    pie.add("", attr, v1, is_label_show=True)
    return pie

In [10]:
age_distribution(app_tab,10001)

In [11]:
# 平均用户留存时间
def avg_time(app_id):
    pass

In [12]:
# 计算某两个app之间的相似度,利用pearson相关系数
from scipy.stats import pearsonr
def get_similarity(id_1,id_2):
    data_1 = app_tab.loc[id_1,['male','_24','25_30','31_35','36_40']].values
    data_2 = app_tab.loc[id_2,['male','_24','25_30','31_35','36_40']].values
    return pearsonr(data_1,data_2)[0]

In [65]:
def cos_like(id_1,id_2):
    data_1 = app_tab.loc[id_1, ['male', '_24', '25_30', '31_35', '36_40']].values.astype('float64')
    data_2 = app_tab.loc[id_2, ['male', '_24', '25_30', '31_35', '36_40']].values.astype('float64')
    num = float(np.matmul(data_1, data_2))
    s = np.linalg.norm(data_1) * np.linalg.norm(data_2)
    return num/s

In [66]:
from sklearn.metrics.pairwise import cosine_similarity
def cos_like_2(id_1,id_2):
    data_1 = app_tab.loc[id_1, ['male', '_24', '25_30', '31_35', '36_40']].values
    data_2 = app_tab.loc[id_2, ['male', '_24', '25_30', '31_35', '36_40']].values
    return cosine_similarity(data_1,data_2)

In [68]:
cos_like(10001,10001)
# app_tab.loc[10001, ['male', '_24', '25_30', '31_35', '36_40']].values.astype('float64')

1.0

In [13]:
def get_app_ids():
    return app_tab.loc[:,'app_id'].values
# 获得和某个app最相似的10个app
def get_most_similar(id_):
    app_ids = get_app_ids()
    distances = {}
    for app_id in app_ids:
        distances[app_id] = get_similarity(app_id,id_)
    most_similar =  sorted(distances.items(),key=lambda item:item[1])[-10:][::-1]
    return most_similar

In [14]:
get_most_similar(10005)

/anaconda3/envs/bellick/lib/python2.7/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


[(341331, nan),
 (191063, 1.0),
 (130508, 1.0),
 (130456, 1.0),
 (331668, 1.0),
 (81223, 1.0),
 (11978, 1.0),
 (130326, 1.0),
 (331647, 1.0),
 (31909, 1.0)]

In [40]:
# 获得某用户的推荐结果
def get_recommender(tel):
    used_apps = user_data.loc[tel].values
    result = {}
    for used_app in used_apps:
        app_id = used_app[0]
        time = used_app[2]
        most_similar = get_most_similar(app_id) # 获得和此app最相似的10个app及其相似度
        for app in most_similar:
            if app[0] in result:
                result[app[0]] += app[1] * time  # 该app的相似度乘以停留时间
            else:
                result[app[0]] = app[1]* time
    sort = sorted(result.items(),key=lambda item:item[1])[-10:][::-1]
    return [item[0] for item in sort]

In [41]:
# user_data.loc[13900001537].values

In [42]:
# app_tab.loc[:,'app_id'].values

In [43]:
import time
start = time.time()
print(get_recommender(13900001537))
end = time.time()
print(end-start)

[331647, 191063, 341331, 130508, 11978, 81223, 31909, 130456, 130326, 331668]
60.3052868843


In [39]:
len(user_data)

64050

In [29]:
len(app_tab)

2017

In [44]:
app_tab.loc[:,'app_id']

app_id
10001      10001
10002      10002
10003      10003
10004      10004
10005      10005
10006      10006
10007      10007
10008      10008
10009      10009
10010      10010
10011      10011
10012      10012
10013      10013
10014      10014
10015      10015
20016      20016
20017      20017
20018      20018
20019      20019
20020      20020
20021      20021
20022      20022
20023      20023
20024      20024
20025      20025
20026      20026
20027      20027
20028      20028
20029      20029
20030      20030
           ...  
242017    242017
270248    270248
180260    180260
270286    270286
270287    270287
180344    180344
330391    330391
50417      50417
240419    240419
330481    330481
200487    200487
200491    200491
80609      80609
270619    270619
270627    270627
270639    270639
270645    270645
240653    240653
240654    240654
270718    270718
270832    270832
10927      10927
270930    270930
10931      10931
240934    240934
50984      50984
200991    200991
241094 